In [ ]:
def print_section(text):
    print('##############################################')
    print('###  ', text)
    print('##############################################')


print_section('Start imports')

import os
import sys
import pathlib
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent.parent)))

import tqdm
import numpy as np
import pickle
import pandas as pd
import matplotlib.cm as cm
import matplotlib.axes as am
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.decomposition
import sklearn.cluster
import sklearn.model_selection

%matplotlib widget

from src.particle_funcs import distance_matrix as distance_matrix
from src.particle_funcs import io as particles_io
from src.leap_funcs.qubo import q_matrix as q_matrix

from src import leap_funcs as leap_funcs
from src.leap_funcs import embedding_quality
from src.leap_funcs.qubo import parameterstudy

from src import h5py_funcs
from src.h5py_funcs import inspections, discoveries, init_custom_getstates, io, parameterstudy_using_info_file

print_section('Finished imports')
#%%

##############################################
###   Start imports
##############################################
no cupy available, imported numpy as usual
Custom getstate functions for dwave.cloud.config.models.ClientConfig, dwave.cloud.client.qpu.Client, dwave.cloud.solver.StructuredSolver, dwave.system.samplers.dwave_sampler.DWaveSampler, dwave.system.composites.embedding.FixedEmbeddingComposite have been initialized.
##############################################
###   Finished imports
##############################################


In [2]:
import io
print('io.DEFAULT_BUFFER_SIZE', io.DEFAULT_BUFFER_SIZE)

io.DEFAULT_BUFFER_SIZE 8192


In [3]:
print_section('Define read from/write to params - Start')
#read_from = ['sample_data', 'pickle']
#write_to = ['pickle', 'None']
read_from = 'pickle'
#write_to = 'pickle'
print('  read_from:', read_from, ' , possibilities: \'sample_data\', \'pickle\'')
#print('  write_to:', write_to, ' , possibilities: \'pickle\', \'None\'')
print_section('Define read from/write to params - Finished')

##############################################
###   Define read from/write to params - Start
##############################################
  read_from: pickle  , possibilities: 'sample_data', 'pickle'
##############################################
###   Define read from/write to params - Finished
##############################################


In [4]:
if read_from == 'pickle':
   print_section('Read from pickle - Start')
   with open(pathlib.Path('../01_out',f'df_7_1.df'), 'rb') as f:
       df = pickle.load(f)
   print_section('Read from pickle - Finished')

##############################################
###   Read from pickle - Start
##############################################
##############################################
###   Read from pickle - Finished
##############################################


In [ ]:
print(df.columns)

In [ ]:
print(df.head().to_string())

In [ ]:
params = df.columns[0:37].drop(['t00', 't01', 't02', 't03', 't04', 't05', 't06', 't07', 't08', 't09', 't10', 't11',\
                                's00', 's01',        's03', 's04',        's06', 's07',        's09', 's10'])
params = df.columns
params_to_drop = [\
       't00', 't01', 't02', 't03', 't04', 't05', 't06', 't07', 't08', 't09', 't10', 't11',
       's00', 's01'       , 's03', 's04',        's06', 's07',        's09', 's10',
       'estimated_runtime', 'num_samples_is_found_best_per_run',
       'is_found_best_per_run', 'num_subs_per_run',
       'num_samples_per_run', 'num_samples_per_sub_per_run', 'num_matched_per_run', 'num_matched_per_sub_per_run',
       'num_samples_matched_per_run', 'submissions',
       'num_samples_matched_per_sub_per_run', 'fraction_samples_is_found_best_per_run'\
]
params_to_scale = [\
       'annealing_time', 'annealing_time_log10', 'programming_thermalization','programming_thermalization_log10',
       'readout_thermalization','readout_thermalization_log10', 'flux_drift_compensation', 'chain_strength',
       'anneal_offsets_1_qubits', 'anneal_offsets_2_qubits',
       'anneal_offsets_3_qubits',
       's00', 's01'       , 's03', 's04',        's06', 's07',        's09', 's10',
]

params = params.drop(params_to_drop)

In [ ]:
param_ranges = [[p.min(), p.max()] for p in df[params.drop('identifiers')].values.T]
param_ranges

In [ ]:
df_params_rescaled = df.copy()
print(df_params_rescaled.head().to_string())
df_params_rescaled[params_to_scale] = df[params_to_scale] / df[params_to_scale].max()
df_params_rescaled.fillna(0, inplace=True)
df_params_rescaled

In [ ]:
df_params_rescaled = df_params_rescaled.sort_values(by='fraction_samples_is_found_best', ascending=False)
df_params_rescaled.head(57)[f'fraction_samples_is_found_best']

In [ ]:
num_chunks = 50
chunks = np.array_split(df_params_rescaled.index, num_chunks)
print('df size:', df_params_rescaled.shape[0])
print('num_chunks:', num_chunks)
print('chunk size approx:', chunks[0].shape[0])
#print('chunk sizes:', chunks)
type(df_params_rescaled.iloc[chunks[0]])
type(df_params_rescaled[params_to_scale].iloc[chunks[0]])

In [ ]:
print(params)
fig, ax = plt.subplots(ncols=1, nrows=num_chunks, figsize=(5, num_chunks*5))
#fig, ax = plt.subplots(figsize=(5, 5))
for i in range(num_chunks):
    ax[i] = sns.boxplot(ax=ax[i], data = df_params_rescaled[params_to_scale].iloc[chunks[i]], order = params_to_scale, orient='v', whis=1.5)
    ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=90)
    #set_xticklabels(params_to_scale, rotation=60)
fig. tight_layout()
fig.savefig(pathlib.Path('out',f'boxplots_params_rescaled_binned_by_fraction_samples_found_best_{num_chunks}bins_{chunks[0].shape[0]}binsize.svg'))
np.arange(5)